In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

### Generate synthetic data

In [2]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 10000
M = 30
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, alpha_range=[0.6, 0.9], beta_range=[0.1, 0.2], 
        class_balance=[0.3, 0.7], seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

### Apply LabelModel

In [3]:
from metal.label_model import LabelModel

lm = LabelModel(seed=2)
lm.train(Ls[0], n_epochs=1000, lr=0.01, print_at=100)
Y_p = lm.score(Ls[1], Ys[1])

Overwriting seed=None to seed=2
Overwriting n_epochs=100 to n_epochs=1000
Reaffirming lr=0.01
Overwriting print_at=10 to print_at=100
[Epoch 0] Loss: 435.422895
[Epoch 100] Loss: 0.004350
[Epoch 200] Loss: 0.001577
[Epoch 300] Loss: 0.001421
[Epoch 400] Loss: 0.001322
[Epoch 500] Loss: 0.001171
[Epoch 600] Loss: 0.000922
[Epoch 700] Loss: 0.000642
[Epoch 800] Loss: 0.000472
[Epoch 900] Loss: 0.000413
[Epoch 999] Loss: 0.000396
Accuracy: 0.780


In [4]:
Y_p = lm.predict(Ls[1])

### Random Search

In [5]:
search_space = {
    'print_at': 200,
    'n_epochs': 1000, # a single constant value
    'l2': [0, 0.1, 0.01], # a list of discrete values
    'lr': {'range': [0.01, 0.0001], 'scale': 'log'}, # a range and scale to interpolate by
}

In [6]:
from metal.tuner import ModelTuner

tuner = ModelTuner(LabelModel)
init_args = []
train_args = [Ls[0]]
model, best_config = tuner.search(init_args, train_args, Ls[1], Ys[1], 
                                  search_space, max_search=5, metric='accuracy')

[1] Testing {'l2': 0.1, 'lr': 0.00014782367342623394}
[Epoch 0] Loss: 348.311680
[Epoch 200] Loss: 2.112149
[Epoch 400] Loss: 1.095611
[Epoch 600] Loss: 0.789488
[Epoch 800] Loss: 0.671334
[Epoch 999] Loss: 0.624715
Accuracy: 0.590
[2] Testing {'l2': 0.01, 'lr': 0.004884765313390349}
[Epoch 0] Loss: 310.110048
[Epoch 200] Loss: 0.064544
[Epoch 400] Loss: 0.063798
[Epoch 600] Loss: 0.063600
[Epoch 800] Loss: 0.063457
[Epoch 999] Loss: 0.063329
Accuracy: 0.752
[3] Testing {'l2': 0, 'lr': 0.00018566433578077888}
[Epoch 0] Loss: 435.422895
[Epoch 200] Loss: 0.824796
[Epoch 400] Loss: 0.282886
[Epoch 600] Loss: 0.124117
[Epoch 800] Loss: 0.059642
[Epoch 999] Loss: 0.031585
Accuracy: 0.683
[4] Testing {'l2': 0.1, 'lr': 0.008167112609961452}
[Epoch 0] Loss: 193.782519
[Epoch 200] Loss: 0.572848
[Epoch 400] Loss: 0.572811
[Epoch 600] Loss: 0.572810
[Epoch 800] Loss: 0.572810
[Epoch 999] Loss: 0.572810
Accuracy: 0.502
[5] Testing {'l2': 0.01, 'lr': 0.0002991777241598981}
[Epoch 0] Loss: 380.069

### Metrics

Calculate metrics in one of two ways.

1. Use metric_score() and pass the metric name
3. The the specific metric's function (e.g., accuracy_score())

In [8]:
from metal.metrics import metric_score, accuracy_score

metric_score(Ys[1], Y_p, 'accuracy')
accuracy_score(Ys[1], Y_p)

0.7782

0.7782

Built-in metrics include:

In [9]:
metric_score(Ys[1], Y_p, 'accuracy')
metric_score(Ys[1], Y_p, 'coverage')
metric_score(Ys[1], Y_p, 'precision')
metric_score(Ys[1], Y_p, 'recall')
metric_score(Ys[1], Y_p, 'f1')
metric_score(Ys[1], Y_p, 'fbeta', beta=1.0)

0.7782

1.0

0.6135135135135135

0.7423152387181164

0.6717963894643385

0.6717963894643385

### Confusion matrix

In [10]:
from metal.analysis import confusion_matrix
confusion_matrix(Y_p, Ys[1])

array([[2270, 1430],
       [ 788, 5512]])

In [11]:
confusion_matrix(Y_p, Ys[1], normalize=True)

array([[0.227 , 0.143 ],
       [0.0788, 0.5512]])

In [12]:
mat = confusion_matrix(Y_p, Ys[1], pretty_print=True)

        y=1    y=2   
 l=1   2270   1430   
 l=2    788   5512   


### Error analysis

In [13]:
from metal.analysis import error_buckets

buckets = error_buckets(Y_p, Ys[1], Ds[1])
tp = buckets[1,1]

In [14]:
tp[:3]

['congress sound represent wall even none within defense successful suffer commercial same college figure base building around station building level pain national card where subject public president some true foot senior fine try scene structure turn legal within so response mr everybody past process',
 'along win allow win hundred evening call model indeed firm window imagine leave arm of beautiful southern issue result challenge six evening major point weight read final fill half nothing wear choice benefit ok purpose such thing risk rate discussion modern second hotel',
 'region about himself room suddenly question threat know effect concern indeed realize two white than effect begin plan oh keep brother degree two pull more pretty training out thus another doctor responsibility degree voice begin responsibility enjoy adult provide recent finger likely finally project use responsibility new finally']